# 逐步提示（问答）

一种名为“逐步回退”提示的提示技术可以通过首先提出“逐步回退”问题来提高对复杂问题的回答效果。这可以与常规的问答应用结合起来，通过对原始问题和逐步回退问题进行检索来实现。

在这里阅读论文[链接](https://arxiv.org/abs/2310.06117)

在这篇优秀的博客文章中，Cobus Greyling对此进行了详细介绍[链接](https://cobusgreyling.medium.com/a-new-prompt-engineering-technique-has-been-introduced-called-step-back-prompting-b00e8954cacb)

在本教程中，我们将复制这种技术。我们稍微修改了所使用的提示，以便更好地适应聊天模型。

In [85]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

In [86]:
# Few Shot Examples
# 定义了几个示例对话
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]

# We now transform these to example messages
# 现在我们将这些转换为示例消息
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [87]:
# 导入ChatPromptTemplate类
from transformers import ChatPromptTemplate

# 创建一个ChatPromptTemplate对象，命名为prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [88]:
# 创建一个问题生成器，使用ChatOpenAI作为输入，设置温度为0，使用StrOutputParser作为输出解析器
question_gen = prompt | ChatOpenAI(temperature=0) | StrOutputParser()

In [182]:
question = "特朗普担任总统期间是否有ChatGPT存在？"

In [183]:
# 调用question_gen对象的invoke方法，传入一个包含问题的字典作为参数
question_gen.invoke({"question": question})

'when was ChatGPT developed?'

In [190]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

search = DuckDuckGoSearchAPIWrapper(max_results=4)


def retriever(query):
    return search.run(query)

In [191]:
# 调用retriever函数，并传入question作为参数
retriever(question)

'This includes content about former President Donald Trump. According to further tests, ChatGPT successfully wrote poems admiring all recent U.S. presidents, but failed when we entered a query for ... On Wednesday, a Twitter user posted screenshots of him asking OpenAI\'s chatbot, ChatGPT, to write a positive poem about former President Donald Trump, to which the chatbot declined, citing it ... While impressive in many respects, ChatGPT also has some major flaws. ... [President\'s Name]," refused to write a poem about ex-President Trump, but wrote one about President Biden ... During the Trump administration, Altman gained new attention as a vocal critic of the president. It was against that backdrop that he was rumored to be considering a run for California governor.'

In [192]:
# 调用question_gen的invoke方法，传入一个包含问题的字典作为参数
# question_gen是一个对象，它有一个invoke方法可以用来生成问题
# 生成的问题会作为参数传递给retriever函数
retriever(question_gen.invoke({"question": question}))

"Will Douglas Heaven March 3, 2023 Stephanie Arnett/MITTR | Envato When OpenAI launched ChatGPT, with zero fanfare, in late November 2022, the San Francisco-based artificial-intelligence company... ChatGPT, which stands for Chat Generative Pre-trained Transformer, is a large language model -based chatbot developed by OpenAI and launched on November 30, 2022, which enables users to refine and steer a conversation towards a desired length, format, style, level of detail, and language. ChatGPT is an artificial intelligence (AI) chatbot built on top of OpenAI's foundational large language models (LLMs) like GPT-4 and its predecessors. This chatbot has redefined the standards of... June 4, 2023 ⋅ 4 min read 124 SHARES 13K At the end of 2022, OpenAI introduced the world to ChatGPT. Since its launch, ChatGPT hasn't shown significant signs of slowing down in developing new..."

In [193]:
# 定义一个模板字符串，用于生成问题回答的提示
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}
{step_back_context}

Original Question: {question}
Answer:"""
# 从模板字符串创建一个对话提示对象
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [203]:
# 导入langchain模块中的hub函数
from langchain import hub

# 设置response_prompt变量为从"langchain-ai/stepback-answer"中获取的模型
response_prompt = hub.pull("langchain-ai/stepback-answer")

In [204]:
chain = (
    {
        # 使用正常问题检索上下文
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # 使用回退问题检索上下文
        "step_back_context": question_gen | retriever,
        # 传递问题
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
)

In [205]:
# 调用chain对象的invoke方法，传入一个字典作为参数
# 字典中包含一个键值对，键为"question"，值为question变量的值
chain.invoke({"question": question})

"No, ChatGPT was not around while Donald Trump was president. ChatGPT was launched on November 30, 2022, which is after Donald Trump's presidency. The context provided mentions that during the Trump administration, Altman, the CEO of OpenAI, gained attention as a vocal critic of the president. This suggests that ChatGPT was not developed or available during that time."

## 基线

In [206]:
# 定义一个字符串模板，用于生成回答问题的提示语
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}

Original Question: {question}
Answer:"""

# 使用字符串模板生成回答问题的提示语对象
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [207]:
# 定义一个变量chain，它是一个由多个步骤组成的管道
chain = (
    {
        # 使用普通问题检索上下文（只使用前3个结果）
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # 传递问题
        "question": lambda x: x["question"],
    }
    | response_prompt  # 添加response_prompt步骤
    | ChatOpenAI(temperature=0)  # 添加ChatOpenAI步骤，并设置temperature为0
    | StrOutputParser()  # 添加StrOutputParser步骤
)

In [208]:
# 使用chain对象调用invoke方法，并传入一个包含键为"question"的字典作为参数
chain.invoke({"question": question})

"Yes, ChatGPT was around while Donald Trump was president. However, it is important to note that the specific context you provided mentions that ChatGPT refused to write a positive poem about former President Donald Trump. This suggests that while ChatGPT was available during Trump's presidency, it may have had limitations or biases in its responses regarding him."